In [ ]:
#| default_exp showdoc

# showdoc
> Display symbol documentation in notebook and website
- order: 8

In [ ]:
#| export
from __future__ import annotations
from nbdev.doclinks import *
from nbdev.config import get_config

from fastcore.docments import *
from fastcore.utils import *

from importlib import import_module
import inspect, sys
from collections import OrderedDict
from textwrap import fill
from types import FunctionType

In [ ]:
#| hide
from inspect import Parameter
from fastcore.test import *

## Documentation For An Object

Render the signature as well as the `docments` to show complete documentation for an object.

In [ ]:
#| export
def _ext_link(url, txt, xtra=""): return f'[{txt}]({url}){{target="_blank" {xtra}}}'

class BasicMarkdownRenderer(MarkdownRenderer):
    "Markdown renderer for `show_doc`"
    def _repr_markdown_(self):
        doc = '---\n\n'
        src = NbdevLookup().code(self.fn)
        if src: doc += _ext_link(src, 'source', 'style="float:right; font-size:smaller"') + '\n\n'
        h = '#'*self.title_level
        doc += f'{h} {self.nm}\n\n'
        return doc+super()._repr_markdown_()

In [ ]:
#| export
def show_doc(sym,  # Symbol to document
             renderer=None,  # Optional renderer (defaults to markdown)
             name:str|None=None,  # Optionally override displayed name of `sym`
             title_level:int=3):  # Heading level to use for symbol name
    "Show signature and docstring for `sym`"
    if renderer is None: renderer = get_config().get('renderer', None)
    if renderer is None: renderer=BasicMarkdownRenderer
    elif isinstance(renderer,str):
        p,m = renderer.rsplit('.', 1)
        renderer = getattr(import_module(p), m)
    if isinstance_str(sym, "Function"): pass
    elif isinstance_str(sym, "TypeDispatch"): pass  # use _str as TypeDispatch will be removed from fastcore
    else:return renderer(sym or show_doc, name=name, title_level=title_level)

You can use `show_doc` to document apis of functions, classes or methods.

### Numpy Docstrings

if you have [numpy docstrings](https://numpydoc.readthedocs.io/en/latest/format.html) instead of `docments`, `show_doc` will attempt to parse and render those just like `docments`.

In [ ]:
#| hide
def f(x=1):
    """
    func docstring in the numpy style.

    This is another line of the docstring.

    Parameters
    ----------
    x : int
        the parameter x

    Returns
    -------
    None
        this function doesn't return anything"""
    ...

show_doc(f)

---

### f

```python

def f(
    x:int=1, # the parameter x
): # this function doesn't return anything


```

*func docstring in the numpy style.*

This is another line of the docstring.

:::{.callout-warning}

Numpy docstring formatting is very strict.  If your docstrings do not strictly adhere to the numpy format, it will not be parsed properly and information about parameters and return values may not properly be rendered in the table below the signature.  Where possible, we recommend using `docments` to annonate your function instead.

:::

## show_doc on Classes

`show_doc` works on Classes, too, including when you use `@patch`.

In [ ]:
#| hide
class Foo:
    def __init__(self, d:str,e:int):
        "This is the docstring for the `__init__` method"
        ...
    @property
    def some_prop(self):
        "This is a class property."
        return 'foo property'

show_doc(Foo)

---

### Foo

```python

def Foo(
    d:str, e:int
):


```

*This is the docstring for the `__init__` method*

You can define methods for the class `Foo` with `@patch` which is convenient in allowing you to break up code for documentation in notebooks.

In [ ]:
#| hide
@patch
def a_method(self:Foo,
             a:list, # param a
             b:dict,c):
        "This is a method"
        ...

_res = show_doc(Foo.a_method)
_res

---

### Foo.a_method

```python

def a_method(
    a:list, # param a
    b:dict, c
):


```

*This is a method*

In [ ]:
#| hide
# signature and docment should show properly when using @patch
assert 'a:list' in str(_res)
assert 'param a' in str(_res)

Class properties also work with showdoc.

In [ ]:
#| hide
show_doc(Foo.some_prop)

---

### Foo.some_prop

```python

def some_prop(
    
):


```

*This is a class property.*

In [ ]:
#| export
def doc(elt):
    "Show `show_doc` info along with link to docs"
    return BasicMarkdownRenderer(elt)

In [ ]:
#| hide
doc(show_doc)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/showdoc.py#L36){target="_blank" style="float:right; font-size:smaller"}

### show_doc

```python

def show_doc(
    sym, # Symbol to document
    renderer:NoneType=None, # Optional renderer (defaults to markdown)
    name:str | None=None, # Optionally override displayed name of `sym`
    title_level:int=3, # Heading level to use for symbol name
):


```

*Show signature and docstring for `sym`*

In [ ]:
#| export
def showdoc_nm(tree):
    "Get the fully qualified name for showdoc."
    return ifnone(patch_name(tree), tree.name)

In [ ]:
#| hide
import ast

In [ ]:
#| hide
code="""
@bar
@patch
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

code2="""
@bar
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

_tree = ast.parse(code).body[0]
test_eq(showdoc_nm(_tree), 'Foo.a_method')

_tree2 = ast.parse(code2).body[0]
test_eq(showdoc_nm(_tree2), 'a_method')

## Other helpers

In [ ]:
#| export
def colab_link(path):
    "Get a link to the notebook at `path` on Colab"
    from IPython.display import Markdown
    cfg = get_config()
    pre = 'https://colab.research.google.com/github/'
    res = f'{pre}{cfg.user}/{cfg.repo}/blob/{cfg.branch}/{cfg.nbs_path.name}/{path}.ipynb'
    display(Markdown(f'[Open `{path}` in Colab]({res})'))

In [ ]:
colab_link('index')

[Open `index` in Colab](https://colab.research.google.com/github/AnswerDotAI/nbdev/blob/main/nbs/index.ipynb)

## Test Edgecases -

In [ ]:
#| hide
from plum import dispatch

In [ ]:
#| hide
@dispatch
def _typ_test(
    a:list, # A list
    b:str, # A second integer
) -> float:
    "Perform op"
    return a.extend(b)

@dispatch
def _typ_test(
    a:str, # An integer
    b:str # A str
) -> float:
    "Perform op"
    return str(a) + b

test_eq(show_doc(_typ_test), None) # show_doc ignores dispatch at the moment

## Export -

In [ ]:
#| hide
import nbdev; nbdev. nbdev_export()